<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/B-Create_API_call_and_Connecting_to_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating of a Flask application showing Citibike stations


In this segment we will create a basic app, where we will connect to the Citibike database, and display the list of stations. 



In [2]:
!pip install -U PyMySQL sqlalchemy flask pyngrok

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 7.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=1bab6e63360503b93ba0931df2d7093381af8cdc08b52ce0370aa347309648d7
  Stored in directory: /home/ubuntu/.cache/pip/wheels/42/46/5e/496d5251f1530ae9988fcd3aad34ad7a46de82d9cc0f61cad6
Successfully built pyngrok


## A Refresher 

Let's remember first how to get data from the database.

In [3]:
# This code creates a connection to the database
from sqlalchemy import create_engine
import pandas as pd

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)

In [7]:
sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017 LIMIT 5"

with engine.connect() as connection:
    stations = pd.read_sql(sql, con=connection)

stations.to_dict(orient='records')

[{'id': 72,
  'name': 'W 52 St & 11 Ave',
  'capacity': 39,
  'lat': 40.7673,
  'lon': -73.9939},
 {'id': 79,
  'name': 'Franklin St & W Broadway',
  'capacity': 33,
  'lat': 40.7191,
  'lon': -74.0067},
 {'id': 82,
  'name': 'St James Pl & Pearl St',
  'capacity': 27,
  'lat': 40.7112,
  'lon': -74.0002},
 {'id': 83,
  'name': 'Atlantic Ave & Fort Greene Pl',
  'capacity': 62,
  'lat': 40.6838,
  'lon': -73.9763},
 {'id': 116,
  'name': 'W 17 St & 8 Ave',
  'capacity': 39,
  'lat': 40.7418,
  'lon': -74.0015}]

## Creating an API endpoint

In [4]:
import os
import threading

from flask import Flask
from pyngrok import ngrok

os.environ["FLASK_DEBUG"] = "true"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
ngrok_authtoken = '2EYf3qVk9mi739HjPwSNZXWAtfy_4jF9NAhGqVVVJmm4YehPW'
ngrok.set_auth_token(ngrok_authtoken)
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url


 * ngrok tunnel 'http://65b3-35-245-232-235.ngrok.io' -> 'http://127.0.0.1:5000'


In [5]:
from flask import Flask, render_template, jsonify
# from flask_ngrok import run_with_ngrok
from sqlalchemy import create_engine
import pandas as pd
import base64
from io import BytesIO
import matplotlib.pyplot as plt

# app = Flask(__name__)
# run_with_ngrok(app)
 
# Setup a connection to the database
conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')
engine = create_engine(conn_string)

#### Now let's start our server:

A few things to notice.

First, notice the `@app.route('/citibike_api',  methods=['GET'])` command. This part specifies that our API endpoint will be accessible under the `http://our_web_server_address/citibike_api`.


Then, we define the function `def citibike_stations():` that will create the response of that API call. What the function returns is what the API call will return back.

Notice that insider the `citibike_stations` function, we connect to the database, and issue an SQL query to the database.

Finally, we get back the results of the query, we put the results in a Python dictionary, and we use the `jsonify` function to convert our dictionary to JSON and return it as the API result.


In [6]:
# Define Flask routes
@app.route("/")
def index():
    return "Hello from Colab!"



In [7]:
@app.route('/citibike_api',  methods=['GET'])
def citibike_stations():

    sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
    # Connect to the database, execute the query, and get back the results
    with engine.connect() as connection:
        stations = pd.read_sql(sql, con=connection)
    
    # Create the response. We will put the retrieved data as a list of
    # dictionaries, under the key "stations".
    list_of_stations = stations.to_dict(orient='records')
    
    api_results = {"stations": list_of_stations}

    # We JSON-ify our dictionary and return it as the API response
    return jsonify(api_results)

In [8]:
@app.route('/station_map',  methods=['GET'])
def station_map():


    # Connect to the database, execute the query, and get back the results
    sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
    with engine.connect() as connection:  
        stations = pd.read_sql(sql, con=connection)

    fig, ax = plt.subplots()
    ax = stations.plot(kind='scatter', x='lon', y='lat', ax=ax)

    buf = BytesIO()
    fig.savefig(buf, format="png")
    # Embed the result in the html output.
    data = base64.b64encode(buf.getbuffer()).decode("ascii")

    # Create the response. We will put the retrieved data as a list of
    # dictionaries, under the key "stations".
    results = {"image": data}

    # We JSON-ify our dictionary and return it as the API response
    return jsonify(results)

    # return f"<img src='data:image/png;base64,{data}'/>"
 


In [9]:


# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": True}).start()